# 会话数据清洗及分析
* 参考示例数据结构

In [57]:
{
  "metrics": {
    "satisfaction": { "value": 85}, #客户满意度从会话中判断，满分100
    "resolution": { "value": 75}, #问题解决程度从会话中判断，满分100
    "attitude": { "value": 95}, #客服态度从会话中判断，满分100
    "risk": { "value": 20} #会话安全程度从会话中判断，满分100，越高越安全
  },
  "summary": "用户咨询Insta360 X4视频误删除恢复问题，客服说明了删除情况下的处理方案，并建议用户使用第三方恢复软件。", #基于会话内容进行摘要总结
  "conversationSummary": {
    "mainIssue": "用户反映相机在室内拍摄太暗，室外拍摄曝光严重", #基于客户会话进行客户主要意图的识别
    "resolutionStatus": {
      "status": "部分解决", #判断客服侧的解决状态，已解决，部分解决，未解决，无法解决
      "description": "提供了参数解释和教程链接" # 分析客服侧的解决方案（摘要关键内容）
    },
    "mainSolution": "解释相机参数（EV、WB、ISO、快门速度）并分享夜景拍摄教程" # 分析客服侧的解决方案，客服的内容信息
  },
  "tags": ["数据恢复", "文件删除", "App操作"], #根据指定的意图标签进行打标签
  "improvementSuggestions": [  #根据客户与客服的对话，给此次对话的客服一些提升建议
    "建议提供更详细的文件恢复指导",
    "可以增加预防性提醒",
    "考虑添加自动备份功能建议"
  ],
  "hotWords": ["InstaX4", "文件恢复", "第三方软件"] #提取本次对话中客户提到的重要关键词
}


{'id': '#16480',
 'agent': '沐沐',
 'customerInfo': {'userId': '827822984@qq.com',
  'device': 'Insta360 X4',
  'history': '首次咨询'},
 'interactionAnalysis': {'totalMessages': 8,
  'agentMessages': 5,
  'userMessages': 3},
 'messages': [{'type': 'system', 'content': '系统：有新的咨询进来了。'},
  {'type': 'agent',
   'content': '您好，感谢您联系影石Insta360，我是沐沐，很高兴为您服务！',
   'sender': '沐沐'},
  {'type': 'user',
   'content': '你好，我在你们ios app上编辑InstaX4 视频时，不小心误操作删除了视频，是通过360X4闪传助手连接iPhone 12 pro max ，内存卡是闪迪 512G extreme pro 。请问你们有方法帮我恢復这条视频吗？',
   'sender': '827822984@qq.com',
   'sentiment': '中立'},
  {'type': 'agent',
   'content': '抱歉给您造成的不便，如果您在相机中删除/格式化文件，是无法找回的，目前并不支持回收站的功能。\n\n如果您在App中删除文件，有以下两种情况：\n1. 相机一直连接着App，在App内编辑文件删除的话，相机内的文件也会被删除掉，无法找回。\n2. 相机连接App，将文件下载/导出到手机后，断开相机与App的连接，再进行编辑删除文件的话，在相机内是有文件备份的。\n另外，建议您下次在删除文件/格式化内存等操作前，先备份好文件哦~',
   'sender': '沐沐'},
  {'type': 'agent', 'content': '辛苦您在相机内查看下是否还有素材的哦', 'sender': '沐沐'},
  {'type': 'user',
   'content': '那你们有提供数据恢复的相关业务吗？',
   'sender': '827822984@

## 1.准备数据

In [58]:
import pymongo
# MongoDB连接配置
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
convoinsight_db = mongo_client["convoinsight"]
conversations_collection = convoinsight_db["conversations"]

In [59]:
import pandas as pd
data = pd.read_excel("./体验家-用户原声-new.xlsx")

In [60]:
data.head()

,会话内容
0,系统: 有新的咨询进来了。\n 沐沐: 您好，欢迎光临影石insta360~我是沐沐，请问有...
1,系统: 有新的咨询进来了。\n 金鱼: 您好，感谢您联系影石Insta360，我是金鱼，很高...
2,系统: 有新的咨询进来了。\n 树叶: 您好，欢迎光临影石insta360~我是树叶，请问有...
3,系统: 有新的咨询进来了。\n 柚子: 您好，感谢您联系影石Insta360，我是柚子，很高...
4,系统: 有新的咨询进来了。\n 15219877537: 手机没办法升级固件怎么办\n 糖米...


In [61]:
data['会话内容'][0]

'系统: 有新的咨询进来了。\n 沐沐: 您好，欢迎光临影石insta360~我是沐沐，请问有什么可以帮到您的吗？\n 15737314172: 安卓手机连接云台吗\n 沐沐: 请问您的手机型号是什么呢\n 15737314172: 荣耀pro\n 15737314172: 荣耀100pro\n 沐沐: 沐沐帮您查看一下，请您稍等下。\n 沐沐: 感谢您的耐心等待，您可以点此链接查看已经测试过可以连接云台并且能够使用功能的机型。\n 值得说明的是，由于安卓/iOS机型众多，以上兼容列表只是包含我们实测过的机型。对列表以外手机，一般只要能安装APP都可以使用，但可能由于性能原因个别功能会有限制，如4K录像、美颜等，球形全景等。感谢您的理解～ \n 温馨提示：除了手机系统需要兼容外，Flow/Flow Pro对手机的尺寸和重量也会有一定的要求哦，适配手机厚度：6.9~10mm；适配手机宽度：64~84mm；适配手机重量：＜300克。\n 沐沐: https://s2-cs-pub-std.kefutoutiao.com/doc/im/tid16480/Flow_Pro兼容性列表1735539287761.pdf\n 15737314172: https://s2-cs-pub-std.kefutoutiao.com/chat_upload/tid16480/1000043380_1735539423_1735539423379_e54a0c.jpg?OSSAccessKeyId=LTAI5tERQ5hZtMsgME6Q1Uap&Expires=1735568256&Signature=zFOL8W99%2FXL%2FUUzYNxUi6y3UHwA%3D\n 15737314172: 荣耀只有这两个可以吗\n '

## 2. 调用代理

In [62]:
import os
# Set the proxy URL and port
proxy_url = 'http://127.0.0.1'
proxy_port = "3566" # !!!please replace it with your own port

# Set the http_proxy and https_proxy environment variables
os.environ['http_proxy'] = f'{proxy_url}:{proxy_port}'
os.environ['https_proxy'] = f'{proxy_url}:{proxy_port}'

## 3. 单条数据:抽取会话内容的基础信息

In [118]:
def system_prompt_for_info_extraction():
    prompt = """请将以下客服会话数据转换成结构化的JSON格式。根据对话内容提取相关信息，并按照以下结构组织：

1. 生成唯一会话ID（如果原始数据中没有明确ID，可使用时间戳或随机字符串）
2. 确定客服人员姓名
3. 提取客户信息（手机号或用户ID）
4. 分析互动情况（总消息数、客服消息数、客户消息数）
5. 按顺序整理所有消息，包括:
   - 消息类型（系统/客服/客户）
   - 消息内容
   - 发送者
   - 对客户消息添加情感分析（正向/中立/负向）

请使用以下JSON结构：

{
  "id": "会话ID",
  "agent": "客服姓名",
  "customerInfo": {
    "userId": "客户ID或手机号",
    "device": "相关设备信息（如有）",
    "history": "客户历史信息（如有）"
  },
  "interactionAnalysis": {
    "totalMessages": 消息总数,
    "agentMessages": 客服消息数,
    "userMessages": 客户消息数
  },
  "messages": [
    {
      "type": "消息类型（system/agent/user）",
      "content": "消息内容",
      "sender": "发送者名称/ID",
      "sentiment": "情感分析（仅用户消息需要）"
    },
    // 更多消息...
  ]
}    
    """
    return prompt

def user_prompt_for_info_extraction(conversations_string):
    prompt = """# 原始会话数据：
{conversations_string}
请根据上述要求将原始会话数据转换为结构化的JSON格式。
    """.format(conversations_string = conversations_string)
    return prompt

In [119]:
print(user_prompt_for_info_extraction("test"))

# 原始会话数据：
test
请根据上述要求将原始会话数据转换为结构化的JSON格式。
    


In [120]:
from openai import OpenAI
import os
# client = OpenAI(api_key=os.getenv('DEEPSEEK_API_KEY'), base_url="https://api.deepseek.com")
client = OpenAI(api_key=os.getenv('OPEN_AI_KEY'))
model_name = "gpt-4o-mini"

In [121]:
import uuid
from datetime import datetime
def custom_length_uuid(length=8):
    """Generate a UUID of custom length (minimum recommended: 8)"""
    # Generate a random UUID and remove hyphens
    u = str(uuid.uuid4()).replace('-', '')
    # Return a substring of the desired length
    return u[:length]

In [122]:
custom_length_uuid()

'318e806d'

In [123]:
def process_single_conversation_extract(conversation:str,test_mode:bool,model="gpt-4o-mini"):
    """
    处理单条评论数据。

    参数:
        conversation: 客服会话对话信息
        test_mode: 是否为测试模式,布尔类型
        model:处理模型版本

    返回:
        Dict[str, Any]: 处理结果字典
    """
    
    try:
        # 准备提示文本
        system_prompt = system_prompt_for_info_extraction()
        user_prompt = user_prompt_for_info_extraction(conversations_string = conversation)

        # 调用模型获取结果
        response = client.chat.completions.create(
            model = model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt },
            ],
            stream=False,
                response_format={
                'type': 'json_object'
            }
        )

        # 获取 token 消耗数量
        token_usage = response.usage.total_tokens if hasattr(response, 'usage') else None

        try:
            content = response.choices[0].message.content
            parsed_content = json.loads(content)  # 解析内容
            # print(parsed_content)
            result_doc = parsed_content
            result_doc['origin_conversation'] = conversation
            result_doc['token_usage'] = token_usage
            result_doc['id'] = "#"+custom_length_uuid()
            result_doc['time'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        except json.JSONDecodeError as e:
            print(f"JSON 解析失败: {e}")
        except AttributeError as e:
            print(f"属性访问失败: {e}")
        except Exception as e:
            print(f"处理过程中发生错误: {e}")
        
        if test_mode:
            return result_doc
        else:
            conversations_collection.insert_one(result_doc)
            print(f"已插入一条数据到conversations集合")
            return result_doc

    except Exception as e:
        # 记录错误信息
        result_doc = {
            'error': str(e),
            "conversation":conversation,
            'processed_at': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'status': 'failed',
            "model": model,
        }
        return result_doc

In [124]:
response = process_single_conversation_extract(conversation=data['会话内容'][1],test_mode=False)

已插入一条数据到conversations集合


In [125]:
data['会话内容'][1]

'系统: 有新的咨询进来了。\n 金鱼: 您好，感谢您联系影石Insta360，我是金鱼，很高兴为您服务！\n 18090636943: 你好为什么我的acepro拍视频会闪烁啊\n 金鱼: 请问您指的闪烁是屏幕闪烁，还是其他的呢\n 18090636943: 录出来的视频\n 18090636943: 摇晃的时候灯光录出来是闪烁的\n 金鱼: 您好，感谢您的反馈！\n 当相机开启“暗光增稳”时，为了减少运动模糊，相机会自动提高快门速度，抗闪烁功能也会随之失效。由于相机快门速度与电视、电脑或其他LED光源的刷新频率不一致，画面便会出现频闪。\n 如您在非夜间运动场景下拍摄，我们建议您可以关闭该功能，并根据环境灯光闪烁频率调整抗闪烁设置，可以改善频闪现象。\n 18090636943: 哦哦\n 18090636943: 谢谢\n 金鱼: 嗯嗯，请问还有其他问题可以帮助到您的吗？\n 金鱼: 客服金鱼发送满意度调查\n 系统: 客户评价为是,非常满意\n '

In [126]:
response

{'id': '#38a4f4a2',
 'agent': '金鱼',
 'customerInfo': {'userId': '18090636943', 'device': None, 'history': None},
 'interactionAnalysis': {'totalMessages': 10,
  'agentMessages': 5,
  'userMessages': 5},
 'messages': [{'type': 'system', 'content': '有新的咨询进来了。', 'sender': '系统'},
  {'type': 'agent',
   'content': '您好，感谢您联系影石Insta360，我是金鱼，很高兴为您服务！',
   'sender': '金鱼'},
  {'type': 'user',
   'content': '你好为什么我的acepro拍视频会闪烁啊',
   'sender': '18090636943',
   'sentiment': '负向'},
  {'type': 'agent', 'content': '请问您指的闪烁是屏幕闪烁，还是其他的呢', 'sender': '金鱼'},
  {'type': 'user',
   'content': '录出来的视频',
   'sender': '18090636943',
   'sentiment': '中立'},
  {'type': 'user',
   'content': '摇晃的时候灯光录出来是闪烁的',
   'sender': '18090636943',
   'sentiment': '负向'},
  {'type': 'agent',
   'content': '您好，感谢您的反馈！当相机开启“暗光增稳”时，为了减少运动模糊，相机会自动提高快门速度，抗闪烁功能也会随之失效。由于相机快门速度与电视、电脑或其他LED光源的刷新频率不一致，画面便会出现频闪。如您在非夜间运动场景下拍摄，我们建议您可以关闭该功能，并根据环境灯光闪烁频率调整抗闪烁设置，可以改善频闪现象。',
   'sender': '金鱼'},
  {'type': 'user',
   'content': '哦哦',
   'sender

## 4. 单条数据：抽取会话内容的诊断信息

In [117]:
def system_prompt_for_info_update():
    prompt = """# 客服对话结构化分析
你是一个有着十年专业经验的客服总监，现在需要基于客户对话进行会话内容的分析和诊断
## 任务描述
分析以下客服与客户的对话内容，将其转化为结构化数据，包括:
1. 客户满意度、问题解决程度、客服态度和安全风险评估等关键指标
2. 对话摘要和主要问题识别
3. 问题解决情况及解决方案
4. 相关标签和关键词提取
5. 客服改进建议

## 分析指南
1. **对话指标评估**:
   - 客户满意度(0-100): 根据客户回应的积极程度、问题解决情况评估
   - 问题解决程度(0-100): 根据客户问题是否得到全面解答评估
   - 客服态度(0-100): 根据客服回应的礼貌度、专业性、响应速度评估
   - 风险评估(0-100): 分析对话中可能存在的安全隐患，数值越高代表越安全
2. **对话摘要**:
   - 提取对话的主要主题和要点，用1-2句话概括
3. **对话详细分析**:
   - 识别客户的主要问题
   - 确定问题解决状态(已解决/部分解决/未解决/无法解决)
   - 概括客服提供的解决方案
4. **标签和关键词**:
   - 标签：根据对话内容提取最匹配的相关标签
   - 关键词：识别客户对话中出现的热门词汇或产品名称
5. **改进建议**:
   - 根据对话分析，提供2-4条客服可以改进的具体建议

## 输出格式
请按以下JSON格式输出分析结果:

```json
{
  "metrics": {
    "satisfaction": { "value": 0-100 },
    "resolution": { "value": 0-100 },
    "attitude": { "value": 0-100 },
    "risk": { "value": 0-100 }
  },
  "summary": "一句话总结对话内容",
  "conversationSummary": {
    "mainIssue": "客户主要问题",
    "resolutionStatus": {
      "status": "已解决/部分解决/未解决/无法解决",
      "description": "解决方案描述"
    },
    "mainSolution": "客服提供的主要解决方案"
  },
  "tags": ["标签1"],
  "improvementSuggestions": [
    "改进建议1",
    "改进建议2",
    "改进建议3"
  ],
  "hotWords": ["关键词1", "关键词2", "关键词3"]
}
```
## 特别注意事项
1. 所有评分必须基于客观事实，避免主观臆断
2. 标签应精准反映对话主题和产品类别
3. 改进建议应具体、可操作且有建设性
4. 如对话未明确表明某信息，应做合理推断并注明
    """
    return prompt

def user_prompt_for_info_update(conversations_string,tags_list_string):
    prompt = """# 原始会话数据：
{conversations_string}
请根据上述要求将原始会话数据转换为结构化的JSON格式。
# 标签的识别请从以下标签进行选择
{tags_list_string}
    """.format(conversations_string = conversations_string,tags_list_string=tags_list_string)
    return prompt

In [97]:
problem_tags_list = """产品使用咨询>功能使用
产品使用咨询>相机操作咨询
产品使用咨询>Insta360 APP咨询
产品使用咨询>Studio 咨询
产品使用咨询>教程需求咨询
产品使用咨询>配件使用
产品使用咨询>云服务使用咨询
产品使用咨询>Connect使用咨询
产品使用咨询>其他产品使用咨询
相机故障问题>镜头外观不良
相机故障问题>镜头功能类不良
相机故障问题>屏外观不良
相机故障问题>屏功能不良
相机故障问题>按键外观/功能不良
相机故障问题>电池外观(功能)不良
相机故障问题>USB盖/喇叭盖/其他盖子损坏
相机故障问题>机身外观不良
相机故障问题>进液
相机故障问题>充电类不良
相机故障问题>发热
相机故障问题>拼接类不良
相机故障问题>Wifi连接类不良
相机故障问题>蓝牙连接不良
相机故障问题>有线连接类不良
相机故障问题>声音类不良
相机故障问题>图像类不良（预览&成片）
相机故障问题>防抖类异常
相机故障问题>死机
相机故障问题>开关机类不良
相机故障问题>升级异常
相机故障问题>续航异常
相机故障问题>相机录制异常/停止录像
相机故障问题>相机文件异常
相机故障问题>内存卡相关异常
相机故障问题>直播异常-专业级
相机故障问题>其他故障
"""

In [98]:
user_prompt_for_info_update(conversations_string = data['会话内容'][0],tags_list_string = problem_tags_list)

'# 原始会话数据：\n系统: 有新的咨询进来了。\n 沐沐: 您好，欢迎光临影石insta360~我是沐沐，请问有什么可以帮到您的吗？\n 15737314172: 安卓手机连接云台吗\n 沐沐: 请问您的手机型号是什么呢\n 15737314172: 荣耀pro\n 15737314172: 荣耀100pro\n 沐沐: 沐沐帮您查看一下，请您稍等下。\n 沐沐: 感谢您的耐心等待，您可以点此链接查看已经测试过可以连接云台并且能够使用功能的机型。\n 值得说明的是，由于安卓/iOS机型众多，以上兼容列表只是包含我们实测过的机型。对列表以外手机，一般只要能安装APP都可以使用，但可能由于性能原因个别功能会有限制，如4K录像、美颜等，球形全景等。感谢您的理解～ \n 温馨提示：除了手机系统需要兼容外，Flow/Flow Pro对手机的尺寸和重量也会有一定的要求哦，适配手机厚度：6.9~10mm；适配手机宽度：64~84mm；适配手机重量：＜300克。\n 沐沐: https://s2-cs-pub-std.kefutoutiao.com/doc/im/tid16480/Flow_Pro兼容性列表1735539287761.pdf\n 15737314172: https://s2-cs-pub-std.kefutoutiao.com/chat_upload/tid16480/1000043380_1735539423_1735539423379_e54a0c.jpg?OSSAccessKeyId=LTAI5tERQ5hZtMsgME6Q1Uap&Expires=1735568256&Signature=zFOL8W99%2FXL%2FUUzYNxUi6y3UHwA%3D\n 15737314172: 荣耀只有这两个可以吗\n \n请根据上述要求将原始会话数据转换为结构化的JSON格式。\n# 标签的识别请从以下标签进行选择\n产品使用咨询>功能使用\n产品使用咨询>相机操作咨询\n产品使用咨询>Insta360 APP咨询\n产品使用咨询>Studio 咨询\n产品使用咨询>教程需求咨询\n产品使用咨询>配件使用\n产品使用咨询>云服务使用咨询\n产品使用咨询>Connect使用咨询\n产品使用咨询>其他产品使用咨询\n相机故障问题>镜头外观不良\n相机故障问题>镜

In [127]:
# 更新会话数据
# 如果文档存在，则更新；如果不存在，则创建新文档
def update_conversation(conversation_id, new_data):
    # 查找现有文档
    existing_doc = conversations_collection.find_one({'id': conversation_id})
    
    # 准备更新数据
    update_data = new_data.copy()
    
    if existing_doc:
        # 更新时间为最新时间
        update_data['times'] = new_data.get('time', existing_doc.get('time'))
        # 合并 token_usage
        if 'token_usage' in new_data and 'token_usage' in existing_doc:
            update_data['token_usage'] = existing_doc['token_usage']+ new_data['token_usage']

        # 合并其他字段，保留现有字段，添加新字段
        for key in existing_doc:
            if key not in update_data and key != '_id':
                update_data[key] = existing_doc[key]
    
    # 执行更新操作
    result = conversations_collection.update_one(
        {'id': conversation_id},
        {'$set': update_data},
        upsert=True  # 如果文档不存在，则创建新文档
    )
    print(f"已更新一条数据到conversations集合")
    return result


In [128]:
def process_single_conversation_update(conversation:str,conversation_id:str,test_mode:bool,model="gpt-4o-mini"):
    """
    处理单条评论数据。

    参数:
        conversation: 客服会话对话信息
        test_mode: 是否为测试模式,布尔类型
        model:处理模型版本

    返回:
        Dict[str, Any]: 处理结果字典
    """
    
    try:
        # 准备提示文本
        system_prompt = system_prompt_for_info_update()
        user_prompt = user_prompt_for_info_update(conversations_string = conversation,tags_list_string = problem_tags_list)

        # 调用模型获取结果
        response = client.chat.completions.create(
            model = model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt },
            ],
            stream=False,
                response_format={
                'type': 'json_object'
            }
        )

        # 获取 token 消耗数量
        token_usage = response.usage.total_tokens if hasattr(response, 'usage') else None

        try:
            content = response.choices[0].message.content
            parsed_content = json.loads(content)  # 解析内容
            # print(parsed_content)
            result_doc = parsed_content
            result_doc['token_usage'] = token_usage
            result_doc['time'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        except json.JSONDecodeError as e:
            print(f"JSON 解析失败: {e}")
        except AttributeError as e:
            print(f"属性访问失败: {e}")
        except Exception as e:
            print(f"处理过程中发生错误: {e}")
        
        if test_mode:
            return result_doc
        else:
            # 更新原始数据
            update_conversation(conversation_id, result_doc)
            return result_doc

    except Exception as e:
        # 记录错误信息
        result_doc = {
            'error': str(e),
            "conversation":conversation,
            'processed_at': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'status': 'failed',
            "model": model,
        }
        return result_doc

In [130]:
response = process_single_conversation_update(conversation=conversations_collection.find()[1]['origin_conversation'],conversation_id="#38a4f4a2",test_mode= False,model="gpt-4o-mini")

已更新一条数据到conversations集合
